In [2]:
import pymongo as pm

In [10]:
conn = pm.MongoClient('localhost', 27017)
db = conn.test
coll = db.users

### insert

In [9]:
db.users.drop()

In [83]:
coll.delete_one({'username':'janedoe'})

In [11]:
from datetime import datetime
user_doc = {'username':'janedoe', 'firstname':'jane', 'surname':'doe',
           'dateofbirth':datetime(1974, 4, 12), 'email':'janedoe74@gmail.com',
           'score':0}
coll.insert_one(user_doc)

### bulk inserts

In [12]:
users_doc = [{'username':'john long', 'firstname':'john', 'surname':'long',
             'dateofbirth':datetime(1976, 5, 15), 'email':'johnlong76@gmail.com',
             'score':4},
            {'username':'wayne rooney', 'firstname':'wayne', 'surname':'rooney',
            'dateofbirth':datetime(1988, 6, 22), 'email':'wayneroo88@gmail.com',
            'score':6},
            {'username':'amber cathy', 'firstname':'amber', 'surname':'cathy',
            'dateofbirth':datetime(1989, 7, 30), 'email':'amber876@gmail.com',
            'score':9},
            {'username':'james wick', 'firstname':'james', 'surname':'wick',
            'dateofbirth':datetime(1990, 12, 1), 'email':'james1201@gmail.com',
            'score':7}]
coll.insert_many(users_doc)

### retrieve single, all

In [13]:
print coll.find_one()

{u'username': u'janedoe', u'surname': u'doe', u'firstname': u'jane', u'dateofbirth': datetime.datetime(1974, 4, 12, 0, 0), u'score': 0, u'_id': ObjectId('58afd4a03753322a2521893d'), u'email': u'janedoe74@gmail.com'}


In [14]:
for a in coll.find():
    print a

{u'username': u'janedoe', u'surname': u'doe', u'firstname': u'jane', u'dateofbirth': datetime.datetime(1974, 4, 12, 0, 0), u'score': 0, u'_id': ObjectId('58afd4a03753322a2521893d'), u'email': u'janedoe74@gmail.com'}
{u'username': u'john long', u'surname': u'long', u'firstname': u'john', u'dateofbirth': datetime.datetime(1976, 5, 15, 0, 0), u'score': 4, u'_id': ObjectId('58afd4a23753322a2521893e'), u'email': u'johnlong76@gmail.com'}
{u'username': u'wayne rooney', u'surname': u'rooney', u'firstname': u'wayne', u'dateofbirth': datetime.datetime(1988, 6, 22, 0, 0), u'score': 6, u'_id': ObjectId('58afd4a23753322a2521893f'), u'email': u'wayneroo88@gmail.com'}
{u'username': u'amber cathy', u'surname': u'cathy', u'firstname': u'amber', u'dateofbirth': datetime.datetime(1989, 7, 30, 0, 0), u'score': 9, u'_id': ObjectId('58afd4a23753322a25218940'), u'email': u'amber876@gmail.com'}
{u'username': u'james wick', u'surname': u'wick', u'firstname': u'james', u'dateofbirth': datetime.datetime(1990, 12

In [27]:
byname = coll.find({'firstname':'jane'}, {'email':1})
for a in byname:
    print a.get('email')

janedoe74@gmail.com


### count

In [28]:
coll.find().count()

5

### sort

In [30]:
# import pymongo as pm
sort1 = coll.find().sort('dateofbirth', pm.DESCENDING)
for a in sort1:
    print a

{u'username': u'james wick', u'surname': u'wick', u'firstname': u'james', u'dateofbirth': datetime.datetime(1990, 12, 1, 0, 0), u'score': 7, u'_id': ObjectId('58afa06337533217738d883d'), u'email': u'james1201@gmail.com'}
{u'username': u'amber cathy', u'surname': u'cathy', u'firstname': u'amber', u'dateofbirth': datetime.datetime(1989, 7, 30, 0, 0), u'score': 9, u'_id': ObjectId('58afa06337533217738d883c'), u'email': u'amber876@gmail.com'}
{u'username': u'wayne rooney', u'surname': u'rooney', u'firstname': u'wayne', u'dateofbirth': datetime.datetime(1988, 6, 22, 0, 0), u'score': 6, u'_id': ObjectId('58afa06337533217738d883b'), u'email': u'wayneroo88@gmail.com'}
{u'username': u'john long', u'surname': u'long', u'firstname': u'john', u'dateofbirth': datetime.datetime(1976, 5, 15, 0, 0), u'score': 4, u'_id': ObjectId('58afa06337533217738d883a'), u'email': u'johnlong76@gmail.com'}
{u'username': u'janedoe', u'surname': u'doe', u'firstname': u'jane', u'dateofbirth': datetime.datetime(1974, 4,

### limit

In [58]:
sort1 = coll.find().sort('score', pm.DESCENDING).limit(3)
for a in sort1:
    print a.get('username'), a.get('score', 0) # score 뒤에 0이 없어도 상관 없음

amber cathy 9
james wick 7
wayne rooney 6


### skip

In [43]:
sort2 = coll.find().sort('score', pm.DESCENDING).limit(3).skip(2) # 가장 앞 2개 결과는 건너뜀
for a in sort2:
    print a.get('username'), a.get('score')

wayne rooney 6
john long 4
janedoe 0


### updating documents1

In [46]:
import copy

In [74]:
old_user = coll.find_one({'username':'janedoe'})
# make a copy
new_user = copy.deepcopy(old_user)

# modify the copy
new_user['email'] = 'janedoe74@example4.com'

# run the update query
# update는 deprecated되었다고 함 / update_one은 먹히지 않음(이건 아래에서 확인가능)
coll.replace_one({'username':'janedoe'}, new_user)

In [75]:
print coll.find_one({'username':'janedoe'})

{u'username': u'janedoe', u'surname': u'doe', u'firstname': u'jane', u'dateofbirth': datetime.datetime(1974, 4, 12, 0, 0), u'score': 1, u'_id': ObjectId('58afa06137533217738d8839'), u'email': u'janedoe74@example4.com'}


### updating documents2 - update_one과 $set을 이용하는 방법

In [77]:
coll.update_one({'username':'janedoe'}, 
            {'$set':{'email':'janedoe74@example323.com'}})

In [78]:
print coll.find_one({'username':'janedoe'})

{u'username': u'janedoe', u'surname': u'doe', u'firstname': u'jane', u'dateofbirth': datetime.datetime(1974, 4, 12, 0, 0), u'score': 1, u'_id': ObjectId('58afa06137533217738d8839'), u'email': u'janedoe74@example323.com'}


### updating documents3 - 변수 여러 개 한 번에 바꾸기

In [86]:
coll.update_many({'username':'janedoe'},
                {'$set':{'email':'janedoe74@example777.com', 'score':1}})

print coll.find_one({'username':'janedoe'})

{u'username': u'janedoe', u'surname': u'doe', u'firstname': u'jane', u'dateofbirth': datetime.datetime(1974, 4, 12, 0, 0), u'score': 1, u'_id': ObjectId('58afa88837533217738d883f'), u'email': u'janedoe74@example777.com'}


### deleting documents

In [87]:
for a in coll.find():
    print a

{u'username': u'john long', u'surname': u'long', u'firstname': u'john', u'dateofbirth': datetime.datetime(1976, 5, 15, 0, 0), u'score': 4, u'_id': ObjectId('58afa06337533217738d883a'), u'email': u'johnlong76@gmail.com'}
{u'username': u'wayne rooney', u'surname': u'rooney', u'firstname': u'wayne', u'dateofbirth': datetime.datetime(1988, 6, 22, 0, 0), u'score': 6, u'_id': ObjectId('58afa06337533217738d883b'), u'email': u'wayneroo88@gmail.com'}
{u'username': u'amber cathy', u'surname': u'cathy', u'firstname': u'amber', u'dateofbirth': datetime.datetime(1989, 7, 30, 0, 0), u'score': 9, u'_id': ObjectId('58afa06337533217738d883c'), u'email': u'amber876@gmail.com'}
{u'username': u'james wick', u'surname': u'wick', u'firstname': u'james', u'dateofbirth': datetime.datetime(1990, 12, 1, 0, 0), u'score': 7, u'_id': ObjectId('58afa06337533217738d883d'), u'email': u'james1201@gmail.com'}
{u'username': u'janedoe', u'surname': u'doe', u'firstname': u'jane', u'dateofbirth': datetime.datetime(1974, 4,

In [88]:
coll.delete_many({'score':1})

for a in coll.find():
    print a

{u'username': u'john long', u'surname': u'long', u'firstname': u'john', u'dateofbirth': datetime.datetime(1976, 5, 15, 0, 0), u'score': 4, u'_id': ObjectId('58afa06337533217738d883a'), u'email': u'johnlong76@gmail.com'}
{u'username': u'wayne rooney', u'surname': u'rooney', u'firstname': u'wayne', u'dateofbirth': datetime.datetime(1988, 6, 22, 0, 0), u'score': 6, u'_id': ObjectId('58afa06337533217738d883b'), u'email': u'wayneroo88@gmail.com'}
{u'username': u'amber cathy', u'surname': u'cathy', u'firstname': u'amber', u'dateofbirth': datetime.datetime(1989, 7, 30, 0, 0), u'score': 9, u'_id': ObjectId('58afa06337533217738d883c'), u'email': u'amber876@gmail.com'}
{u'username': u'james wick', u'surname': u'wick', u'firstname': u'james', u'dateofbirth': datetime.datetime(1990, 12, 1, 0, 0), u'score': 7, u'_id': ObjectId('58afa06337533217738d883d'), u'email': u'james1201@gmail.com'}


### operations on sub-documents embedded in a list

In [126]:
user_doc1 = {'username':'foouser',
            'emails':[{'email':'foouser1@example.com', 'primary':True},
                     {'email':'foouser2@example.com', 'primary':False},
                     {'email':'foouser3@example.com', 'primary':False}
                     ]
            }

coll.insert(user_doc1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


ObjectId('58afafca37533217738d8843')

In [127]:
for a in coll.find():
    print a

{u'username': u'john long', u'surname': u'long', u'firstname': u'john', u'dateofbirth': datetime.datetime(1976, 5, 15, 0, 0), u'score': 4, u'_id': ObjectId('58afa06337533217738d883a'), u'email': u'johnlong76@gmail.com'}
{u'username': u'wayne rooney', u'surname': u'rooney', u'firstname': u'wayne', u'dateofbirth': datetime.datetime(1988, 6, 22, 0, 0), u'score': 6, u'_id': ObjectId('58afa06337533217738d883b'), u'email': u'wayneroo88@gmail.com'}
{u'username': u'amber cathy', u'surname': u'cathy', u'firstname': u'amber', u'dateofbirth': datetime.datetime(1989, 7, 30, 0, 0), u'score': 9, u'_id': ObjectId('58afa06337533217738d883c'), u'email': u'amber876@gmail.com'}
{u'username': u'james wick', u'surname': u'wick', u'firstname': u'james', u'dateofbirth': datetime.datetime(1990, 12, 1, 0, 0), u'score': 7, u'_id': ObjectId('58afa06337533217738d883d'), u'email': u'james1201@gmail.com'}
{u'username': u'foouser', u'_id': ObjectId('58afafca37533217738d8843'), u'emails': [{u'email': u'foouser1@examp

In [99]:
from bson.objectid import ObjectId

In [124]:
coll.delete_one({'_id':ObjectId('58afad6037533217738d8842')})

### $pull

In [128]:
#coll.update({'username':'foouser'}, {'emails':{'email':'foouser2@example.com'}})
# 이렇게 하면 emails 안의 email에 foouser2@example.com만 남게 된다

# use $pull to remove 'foouser2@example.com' email sub-document
# update는 deprecated됨
coll.update_one({'username':'foouser'}, {'$pull':{'emails':{'email':'foouser2@example.com'}}})

In [132]:
print coll.find_one({'username':'foouser'})

{u'username': u'foouser', u'_id': ObjectId('58afafca37533217738d8843'), u'emails': [{u'email': u'foouser1@example.com', u'primary': True}, {u'email': u'foouser3@example.com', u'primary': False}]}


### $ne

In [133]:
# primary가 True가 아닌 것들만 제외(당겨라, pull)
coll.update_many({'username':'foouser'}, {'$pull':{'emails':{'primary':{'$ne':True}}}})

In [135]:
coll.find_one({'username':'foouser'})

{u'_id': ObjectId('58afafca37533217738d8843'),
 u'emails': [{u'email': u'foouser1@example.com', u'primary': True}],
 u'username': u'foouser'}

### $push

In [136]:
# append a new email sub-document to the user document
new_email = {'email':'foouser4@example.com', 'primary':False}
coll.update({'username':'foouser'}, {'$push':{'emails':{'email':new_email}}})

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  app.launch_new_instance()


{u'n': 1, u'nModified': 1, u'ok': 1.0, 'updatedExisting': True}

In [137]:
coll.find_one({'username':'foouser'})

{u'_id': ObjectId('58afafca37533217738d8843'),
 u'emails': [{u'email': u'foouser1@example.com', u'primary': True},
  {u'email': {u'email': u'foouser4@example.com', u'primary': False}}],
 u'username': u'foouser'}

### $set

In [239]:
# sub document의 첫 번째 {}는 내용이 바뀌지가 않음
user_doc2 = {
  "username":"foouser22",
  "emails":[
    {
      "email":"foouser1@example.com",
      "primary":123
    },{
      "email":"foouser2@example2.com",
      "primary":False
    },{ 
      "email":"foouser3@example3.com",
      "primary":False
    }
  ]
}

In [240]:
coll.insert_one(user_doc2)

In [241]:
'''for a in coll.find():
    print a'''
coll.find_one({'username':'foouser22'})

{u'_id': ObjectId('58afd11037533217738d8847'),
 u'emails': [{u'email': u'foouser1@example.com', u'primary': 123},
  {u'email': u'foouser2@example2.com', u'primary': False},
  {u'email': u'foouser3@example3.com', u'primary': False}],
 u'username': u'foouser22'}

In [238]:
coll.delete_one({'_id':ObjectId('58afcfa137533217738d8846')})

In [246]:
coll.update_one({'emails.email':'foouser1@example.com'}, {'$set':{'emails.$.primary':47657}})

In [248]:
coll.update_one({'emails.email':'foouser2@example2.com'}, {'$set':{'emails.$.primary':55}})

In [234]:
coll.update_one({'emails.email':'foouser3@example3.com'}, {'$set':{'emails.$.primary':True}})

In [225]:
'''coll.update_one({"emails.email":"foouser1@example.com"},
  {"$set":{"emails.$.primary":False}})'''

In [249]:
coll.find_one({'username':'foouser22'})

{u'_id': ObjectId('58afd11037533217738d8847'),
 u'emails': [{u'email': u'foouser1@example.com', u'primary': 123},
  {u'email': u'foouser2@example2.com', u'primary': 55},
  {u'email': u'foouser3@example3.com', u'primary': False}],
 u'username': u'foouser22'}

In [15]:
# 원래는 터미널에서 실행하는 명령어
# 파이썬 내에서 사용하려면 !를 붙이면 된다
!mongoexport -d test -c users --out mydb.json

2017-02-24T15:33:13.912+0900	connected to: localhost
2017-02-24T15:33:13.913+0900	exported 6 records


In [17]:
!mongorestore -d mydb ./dump/mydb

2017-02-24T15:33:18.800+0900	the --db and --collection args should only be used when restoring from a BSON file. Other uses are deprecated and will not exist in the future; use --nsInclude instead
2017-02-24T15:33:18.801+0900	Failed: mongorestore target './dump/mydb' invalid: stat ./dump/mydb: no such file or directory


In [15]:
for a in coll.find():
    print a

{u'username': u'janedoe', u'surname': u'doe', u'firstname': u'jane', u'dateofbirth': datetime.datetime(1974, 4, 12, 0, 0), u'score': 0, u'_id': ObjectId('58afd4a03753322a2521893d'), u'email': u'janedoe74@gmail.com'}
{u'username': u'john long', u'surname': u'long', u'firstname': u'john', u'dateofbirth': datetime.datetime(1976, 5, 15, 0, 0), u'score': 4, u'_id': ObjectId('58afd4a23753322a2521893e'), u'email': u'johnlong76@gmail.com'}
{u'username': u'wayne rooney', u'surname': u'rooney', u'firstname': u'wayne', u'dateofbirth': datetime.datetime(1988, 6, 22, 0, 0), u'score': 6, u'_id': ObjectId('58afd4a23753322a2521893f'), u'email': u'wayneroo88@gmail.com'}
{u'username': u'amber cathy', u'surname': u'cathy', u'firstname': u'amber', u'dateofbirth': datetime.datetime(1989, 7, 30, 0, 0), u'score': 9, u'_id': ObjectId('58afd4a23753322a25218940'), u'email': u'amber876@gmail.com'}
{u'username': u'james wick', u'surname': u'wick', u'firstname': u'james', u'dateofbirth': datetime.datetime(1990, 12

In [19]:
for a in coll.find({'score':{'$gt':8}}):
    print a

{u'username': u'amber cathy', u'surname': u'cathy', u'firstname': u'amber', u'dateofbirth': datetime.datetime(1989, 7, 30, 0, 0), u'score': 9, u'_id': ObjectId('58afd4a23753322a25218940'), u'email': u'amber876@gmail.com'}


In [22]:
bb = {}
def haha():    
    for a in coll.find({'score':{'$lt':4}}):
        bb = a
        return bb

In [25]:
abc = haha()
abc

{u'_id': ObjectId('58afd4a03753322a2521893d'),
 u'dateofbirth': datetime.datetime(1974, 4, 12, 0, 0),
 u'email': u'janedoe74@gmail.com',
 u'firstname': u'jane',
 u'score': 0,
 u'surname': u'doe',
 u'username': u'janedoe'}